In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model, Sequential
import os

In [0]:
DIR = 'drive/My Drive/animals10/raw-img/'

In [3]:
datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split=0.2
)
train_generator = datagen.flow_from_directory(
    DIR,
    target_size=(160, 160),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    subset='training'
)
validation_generator = datagen.flow_from_directory(
    DIR,
    target_size=(160, 160),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    subset='validation'
)

Found 20947 images belonging to 10 classes.
Found 5232 images belonging to 10 classes.


In [4]:
base_model = MobileNetV2(input_shape=(160, 160, 3),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False

9412608/9406464 [==============================] - 0s 0us/step


In [0]:
global_average_layer = GlobalAveragePooling2D()
prediction_layer = Dense(10, activation='softmax')
model = Sequential([
    base_model, 
    global_average_layer, 
    prediction_layer
])
base_learning_rate = 0.001
model.compile(optimizer=RMSprop(lr=base_learning_rate),
              loss=BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                12810     
Total params: 2,270,794
Trainable params: 12,810
Non-trainable params: 2,257,984
_________________________________________________________________


In [0]:
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // 32,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32,
    epochs=10,
    callbacks=[cp_callback]
)

Epoch 1/10
654/654 [==============================] - ETA: 0s - loss: 0.1273 - accuracy: 0.7478 
Epoch 00001: saving model to training_1/cp.ckpt
654/654 [==============================] - 15450s 24s/step - loss: 0.1273 - accuracy: 0.7478 - val_loss: 0.0627 - val_accuracy: 0.8850
Epoch 2/10
654/654 [==============================] - ETA: 0s - loss: 0.0523 - accuracy: 0.9052
Epoch 00002: saving model to training_1/cp.ckpt
654/654 [==============================] - 91s 139ms/step - loss: 0.0523 - accuracy: 0.9052 - val_loss: 0.0496 - val_accuracy: 0.9095
Epoch 3/10
654/654 [==============================] - ETA: 0s - loss: 0.0411 - accuracy: 0.9264
Epoch 00003: saving model to training_1/cp.ckpt
654/654 [==============================] - 92s 140ms/step - loss: 0.0411 - accuracy: 0.9264 - val_loss: 0.0448 - val_accuracy: 0.9191
Epoch 4/10
654/654 [==============================] - ETA: 0s - loss: 0.0351 - accuracy: 0.9362
Epoch 00004: saving model to training_1/cp.ckpt
654/654 [===========